In [55]:
import pandas as pd

In [56]:
df = pd.read_csv('appartments.csv')

In [57]:
df.dropna(subset=["lng", "lat"], inplace=True)
df

,city,media_count,immo_id,district,title,url,time_dest2,time_dest3,time_dest,rent,sqm,address,lat,contact_name,zip_code,lng,rooms
1,Berlin,7,93140523,Zehlendorf (Zehlendorf),Erstbezug nach Modernisierung,https://www.immobilienscout24.de/expose/93140523,17.333333,NaN,39.516667,"632,73","62,39","Argentinische Allee 98, Zehlendorf (Zehlendorf...",52.44925,Herr Dominic Sydow,14163,13.24860,"2,5"
2,Berlin,9,93136519,Lichterfelde (Steglitz),Gut geschnittene Wohnung am südlichen Stadtrand!,https://www.immobilienscout24.de/expose/93136519,15.033333,NaN,37.333333,"444,52","77,12","Réaumurstr. 22, Lichterfelde (Steglitz), Berlin",52.41217,Frau Dominique Hartstock,12207,13.31527,"2,5"
3,Berlin,10,92213120,Zehlendorf (Zehlendorf),Achtung!+++ 3 Zimmer Wohnung in Zehlendorf mit...,https://www.immobilienscout24.de/expose/92213120,10.550000,NaN,34.016667,"723,01","85,06","Sundgauer Straße 47, Zehlendorf (Zehlendorf), ...",52.44151,Frau Patricia Köhler,14169,13.27058,3
4,Berlin,3,93120442,Lichterfelde (Steglitz),Schöne Familienwohnung!,https://www.immobilienscout24.de/expose/93120442,15.150000,NaN,33.416667,"720,30","95,05","Celsiusstr. 52, Lichterfelde (Steglitz), Berlin",52.41215,Herr Johann-Owe Schuldt,12207,13.31114,"3,5"
6,Berlin,8,93082388,Lichterfelde (Steglitz),"Saniert, mit Einbauküche, Duschbad, Laminat u....",https://www.immobilienscout24.de/expose/93082388,5.983333,NaN,32.983333,"638,71","63,68","Hindenburgdamm 23, Lichterfelde (Steglitz), Be...",52.43780,Service-Center,12203,13.31569,"2,5"
7,Berlin,0,93052554,Lichterfelde (Steglitz),"Wohnung mit WBS, Wohnen in Berlin-Lichterfelde",https://www.immobilienscout24.de/expose/93052554,10.516667,NaN,27.316667,"625,20","87,44","Morgensternstr. 14, Lichterfelde (Steglitz), ...",52.42582,Frau Ekaterini Fränkel,12207,13.31783,3
8,Berlin,14,93011666,Zehlendorf (Zehlendorf),Ruhige 3-Zimmerwohnung mit Balkon im Grünen,https://www.immobilienscout24.de/expose/93011666,14.400000,NaN,31.900000,799,"88,53","Gartenstraße 4a, Zehlendorf (Zehlendorf), Berlin",52.43288,Andrea Gärtner,14169,13.26217,3
9,Berlin,9,91923586,Lichterfelde (Steglitz),**Charmante 3-Zimmer Balkonwohnung mit EBK und...,https://www.immobilienscout24.de/expose/91923586,12.050000,NaN,45.500000,710,"78,08","Mürwikerstr. 6, Lichterfelde (Steglitz), Berlin",52.42597,Herr Vincent Mochow,14167,13.27845,3
11,Berlin,0,92946780,Lichterfelde (Steglitz),Wohnen in kleiner Eigentumswohnanlage,https://www.immobilienscout24.de/expose/92946780,15.033333,NaN,46.083333,600,"79,91","Achenseeweg 69/71, Lichterfelde (Steglitz), Be...",52.41107,Herr Kristek,12209,13.33482,3
12,Berlin,11,92923277,Lichterfelde (Steglitz),"Schöne 2,5-Zimmer Wohnung mit Balkon",https://www.immobilienscout24.de/expose/92923277,12.216667,NaN,37.733333,600,"66,63","Ostpreußendamm 122a, Lichterfelde (Steglitz), ...",52.41869,Firma Klaus-Peter Hausverwaltung UG (haftungs...,12207,13.30601,"2,5"


In [70]:
df["avg_time"] = 0.5*(df.time_dest + df.time_dest2)
df.sort_values("avg_time", inplace=True)
top10=df.head(n=10)
top10

,city,media_count,immo_id,district,title,url,time_dest2,time_dest3,time_dest,rent,sqm,address,lat,contact_name,zip_code,lng,rooms,avg_time
28,Berlin,11,38971160,Lichterfelde (Steglitz),Zentral gelegen am Botanischen Garten,https://www.immobilienscout24.de/expose/38971160,6.866667,NaN,28.083333,590,80,"Unter den Eichen 97, Lichterfelde (Steglitz), ...",52.44566,Herr Kübler Selectra Kübler GmbH Berlin,12203,13.29466,"2,5",17.475000
27,Berlin,3,84483254,Lichterfelde (Steglitz),Zentral am Botanischen Garten 3 Zimmer mit gro...,https://www.immobilienscout24.de/expose/84483254,6.866667,NaN,28.083333,800,93,"Unter den Eichen 97, Lichterfelde (Steglitz), ...",52.44566,Herr Kübler Selectra Kübler GmbH Berlin,12203,13.29466,3,17.475000
24,Berlin,5,69129400,Lichterfelde (Steglitz),"Wohnung mit WBS, Wohnen in Berlin-Lichterfelde",https://www.immobilienscout24.de/expose/69129400,10.516667,NaN,27.316667,"642,64","89,88","Morgensternstr. 14, Lichterfelde (Steglitz), ...",52.42582,Frau Ekaterini Fränkel,12207,13.31783,3,18.916667
7,Berlin,0,93052554,Lichterfelde (Steglitz),"Wohnung mit WBS, Wohnen in Berlin-Lichterfelde",https://www.immobilienscout24.de/expose/93052554,10.516667,NaN,27.316667,"625,20","87,44","Morgensternstr. 14, Lichterfelde (Steglitz), ...",52.42582,Frau Ekaterini Fränkel,12207,13.31783,3,18.916667
36,Berlin,0,93052554,Lichterfelde (Steglitz),"Wohnung mit WBS, Wohnen in Berlin-Lichterfelde",https://www.immobilienscout24.de/expose/93052554,10.516667,NaN,27.316667,"625,20","87,44","Morgensternstr. 14, Lichterfelde (Steglitz), ...",52.42582,Frau Ekaterini Fränkel,12207,13.31783,3,18.916667
21,Berlin,4,92142214,Lichterfelde (Steglitz),3-Zimmer-Wohnung in Steglitz zu vermieten. WBS...,https://www.immobilienscout24.de/expose/92142214,7.366667,NaN,30.666667,"505,24","69,61","Drakestraße 55a, Lichterfelde (Steglitz), Berlin",52.44048,Frau Ayten Hennig,12205,13.29884,3,19.016667
42,Berlin,13,92602369,Zehlendorf (Zehlendorf),3-Zimmer Wohnung mit Einbauküche in Zehlendorf,https://www.immobilienscout24.de/expose/92602369,9.300000,NaN,29.600000,650,65,"Berliner Straße 52, Zehlendorf (Zehlendorf), B...",52.43822,NaN,14169,13.27210,3,19.450000
13,Berlin,13,92602369,Zehlendorf (Zehlendorf),3-Zimmer Wohnung mit Einbauküche in Zehlendorf,https://www.immobilienscout24.de/expose/92602369,9.300000,NaN,29.600000,650,65,"Berliner Straße 52, Zehlendorf (Zehlendorf), B...",52.43822,NaN,14169,13.27210,3,19.450000
6,Berlin,8,93082388,Lichterfelde (Steglitz),"Saniert, mit Einbauküche, Duschbad, Laminat u....",https://www.immobilienscout24.de/expose/93082388,5.983333,NaN,32.983333,"638,71","63,68","Hindenburgdamm 23, Lichterfelde (Steglitz), Be...",52.43780,Service-Center,12203,13.31569,"2,5",19.483333
35,Berlin,8,93082388,Lichterfelde (Steglitz),"Saniert, mit Einbauküche, Duschbad, Laminat u....",https://www.immobilienscout24.de/expose/93082388,5.983333,NaN,32.983333,"638,71","63,68","Hindenburgdamm 23, Lichterfelde (Steglitz), Be...",52.43780,Service-Center,12203,13.31569,"2,5",19.483333


In [66]:
# see https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Quickstart.ipynb
import folium
print(folium.__version__)

0.2.1


In [107]:
map = folium.Map(location=[52.520645, 13.409779])
# see http://deparkes.co.uk/2016/06/24/folium-marker-clusters/
marker_cluster = folium.MarkerCluster("appartments").add_to(map)

for row in top10.itertuples():
    html = 'Average travel time: {0:.2f} min </br> <a target="_blank" href="{1}">{2}</a>'.format(row.avg_time, row.url, row.title)
    iframe = folium.element.IFrame(html=html.decode("utf8"), width=300, height=100)
    popup = folium.Popup(iframe, max_width=300)
    folium.Marker([row.lat, row.lng], popup=popup).add_to(marker_cluster)


In [108]:
# see https://nbviewer.jupyter.org/github/ocefpaf/folium_notebooks/blob/master/test_fit_bounds.ipynb
map.fit_bounds(map.get_bounds())
map